In [1]:
import pandas as pd

In [2]:
ilevel4 = pd.read_csv('../source/fixtures/ddf--datapoints--ilevels4--by--country--time.csv')
ilevel4_wb = pd.read_csv('../source/fixtures/ddf--datapoints--ilevels4_wb--by--country--time.csv')
ilevel4_latest = pd.read_csv('../source/fixtures/wb_groups.csv')

In [9]:
ilevel4['ilevels4'].unique()

array(['Level 1', 'Level 2', 'Level 3', 'Level 4'], dtype=object)

In [ ]:
ilevel4_wb['ilevels4_wb'].unique()

array(['Low income', 'Lower middle income', 'Upper middle income',
       'High income'], dtype=object)

In [3]:
m1 = { "Level 1": 'lic', 'Level 2': 'lmic', 'Level 3': 'umic', 'Level 4': 'hic'}
m2 = {'Low income': 'lic', 'Lower middle income': 'lmic', 'Upper middle income': 'umic', 'High income': 'hic'}

In [42]:
ilevel4['ilevels4'] = ilevel4['ilevels4'].map(m1)
ilevel4_wb['ilevels4_wb'] = ilevel4_wb['ilevels4_wb'].map(m2)

In [43]:
ilevel4_latest

,country,time,wb
0,afg,1800,lic
1,afg,1801,lic
2,afg,1802,lic
3,afg,1803,lic
4,afg,1804,lic
...,...,...,...
48940,zwe,2046,lmic
48941,zwe,2047,lmic
48942,zwe,2048,lmic
48943,zwe,2049,lmic


In [7]:
ilevel4_latest.columns = ['country', 'name', 'time', 'gm', 'wb']
ilevel4_latest = ilevel4_latest.set_index(['country', 'time'])['wb']

In [44]:
ilevel4

,country,time,ilevels4
0,afg,1800,lic
1,afg,1801,lic
2,afg,1802,lic
3,afg,1803,lic
4,afg,1804,lic
...,...,...,...
48940,zwe,2046,lmic
48941,zwe,2047,lmic
48942,zwe,2048,lmic
48943,zwe,2049,lmic


In [45]:
ilevel4_wb

,country,time,ilevels4_wb
0,afg,1987,lic
1,afg,1988,lic
2,afg,1989,lic
3,afg,1990,lic
4,afg,1991,lic
...,...,...,...
6656,zwe,2016,lic
6657,zwe,2017,lic
6658,zwe,2018,lmic
6659,zwe,2019,lmic


In [9]:
ilevel4_latest = ilevel4_latest.map(m2)

In [11]:
ilevel4_latest.hasnans

False

In [12]:
ilevel4_latest = ilevel4_latest.reset_index()

In [13]:
df = pd.read_csv('../../poverty_rates/ddf--datapoints--poverty_population--by--country--time--wb_level.csv')

In [14]:
df.head()

,country,time,wb_level,poverty_population
0,afg,1800,wb_lvl1,3045636
1,afg,1800,wb_lvl2,158538
2,afg,1800,wb_lvl3,51359
3,afg,1800,wb_lvl4,16832
4,afg,1800,wb_lvl5,7636


In [15]:
df1 = df[df.wb_level == 'wb_lvl1']

In [16]:
df1 = df1[['country', 'time', 'poverty_population']].set_index(['country', 'time'])

In [17]:
df1

poverty_population
country time                    
afg     1800             3045636
        1801             3045636
        1802             3045636
        1803             3045636
        1804             3045636
...                          ...
zwe     2046             6022805
        2047             5445183
        2048             5509108
        2049             5578022
        2050             5646500

[48945 rows x 1 columns]

In [46]:
gm = ilevel4.set_index(['country', 'time'])
wb = ilevel4_wb.set_index(['country', 'time'])

In [27]:
gm.to_csv('tmp_gm.csv')

In [21]:
df1.shape

(48945, 1)

In [22]:
gm.shape

(48945, 1)

In [47]:
wb.columns = ['ilevels4']

In [48]:
gm.update(wb)

In [49]:
gm

ilevels4
country time         
afg     1800      lic
        1801      lic
        1802      lic
        1803      lic
        1804      lic
...               ...
zwe     2046     lmic
        2047     lmic
        2048     lmic
        2049     lmic
        2050     lmic

[48945 rows x 1 columns]

In [50]:
gm.to_csv('tmp_gm_2.csv')

In [36]:
grps = gm.copy()

In [20]:
grps = ilevel4_latest.set_index(['country', 'time']).copy()

In [21]:
grps

wb
country time      
afg     1800   lic
        1801   lic
        1802   lic
        1803   lic
        1804   lic
...            ...
zwe     2046  lmic
        2047  lmic
        2048  lmic
        2049  lmic
        2050  lmic

[48945 rows x 1 columns]

In [51]:
grps.to_csv('hisham.csv')

In [22]:
df1['grp'] = grps['wb']

In [23]:
df1['grp'].hasnans

False

In [24]:
df1 = df1.reset_index()

In [25]:
df1

,country,time,poverty_population,grp
0,afg,1800,3045636,lic
1,afg,1801,3045636,lic
2,afg,1802,3045636,lic
3,afg,1803,3045636,lic
4,afg,1804,3045636,lic
...,...,...,...,...
48940,zwe,2046,6022805,lmic
48941,zwe,2047,5445183,lmic
48942,zwe,2048,5509108,lmic
48943,zwe,2049,5578022,lmic


In [26]:
res = df1.groupby(['grp', 'time'])['poverty_population'].sum()

In [27]:
res

grp   time
hic   1944          78
      1945          67
      1946          48
      1947          27
      1948          16
                ...   
umic  2046    70499082
      2047    68633841
      2048    68588950
      2049    61199947
      2050    58233999
Name: poverty_population, Length: 774, dtype: int64

In [28]:
res = res.reset_index()

In [29]:
gs = res['grp'].unique()

In [30]:
gs

array(['hic', 'lic', 'lmic', 'umic'], dtype=object)

In [31]:
gt = pd.CategoricalDtype(['lic', 'lmic', 'umic', 'hic'], ordered=True)

In [32]:
res['grp'] = res['grp'].astype(gt)

In [33]:
res

,grp,time,poverty_population
0,hic,1944,78
1,hic,1945,67
2,hic,1946,48
3,hic,1947,27
4,hic,1948,16
...,...,...,...
769,umic,2046,70499082
770,umic,2047,68633841
771,umic,2048,68588950
772,umic,2049,61199947


In [34]:
res = res.sort_values(by=['grp', 'time'])

In [35]:
res.columns = ['income_groups', 'time', 'poverty_population_under_1_9_per_day']

In [36]:
res

,income_groups,time,poverty_population_under_1_9_per_day
107,lic,1800,772199533
108,lic,1801,774849801
109,lic,1802,777350868
110,lic,1803,779956143
111,lic,1804,782794118
...,...,...,...
102,hic,2046,8366171
103,hic,2047,8301898
104,hic,2048,7674482
105,hic,2049,12912499


In [37]:
m = {'lic': 'low_income', 'hic': 'high_income', 'lmic': 'lower_middle_income', 'umic': 'upper_middle_income'}

In [38]:
res['income_groups'] = res['income_groups'].map(m)

In [39]:
res

,income_groups,time,poverty_population_under_1_9_per_day
107,low_income,1800,772199533
108,low_income,1801,774849801
109,low_income,1802,777350868
110,low_income,1803,779956143
111,low_income,1804,782794118
...,...,...,...
102,high_income,2046,8366171
103,high_income,2047,8301898
104,high_income,2048,7674482
105,high_income,2049,12912499


In [40]:
res.dtypes['income_groups']

CategoricalDtype(categories=['low_income', 'lower_middle_income', 'upper_middle_income',
                  'high_income'],
, ordered=True)

In [41]:
res.to_csv('../../poverty_rates/ddf--datapoints--poverty_population_under_1_9_per_day--by--income_groups--time.csv', index=False)